<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/WebUI_Marbal_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thanks for your contributions

藏經閣
https://drive.google.com/drive/folders/1vGc16Bb8CDW1piUj_5thzbsCmDFamYyt

In [ ]:
#@title Setup Tool to use models on Marbel Drive

# (optional: check GPU)
!nvidia-smi

# define working directory
root_dir = "/content"
tool_dir = "/content/stable-diffusion-webui"

%cd $root_dir

# Load Google Drive for my models
from google.colab import drive
drive.mount('/content/drive')

# 685f96: The version most commonly used to merge models
version = 'latest' #@param ["latest", "685f9631b56ff8bd43bce24ff5ce0f9a0e9af490"] {allow-input: true}

def use_cuda():
  # use cuda instead of cpu for large model (e.g., 7G)
  !sed -i 's/weight_load_location = None if cmd_opts.lowram else "cpu"/weight_load_location = None if cmd_opts.lowram else "cuda"/g' /content/stable-diffusion-webui/modules/shared.py

def install_dep():
  # install xformers
  %cd /content
  !pip install xformers
  # !python -m xformers.info

def install_extentions():
  %cd {tool_dir}/extensions
  
  !git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git
  !git clone https://github.com/bbc-mc/sdweb-merge-board
  !git clone https://github.com/tkalayci71/embedding-inspector
  !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger
  !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui
  !git clone https://github.com/Malisius/booru2prompt.git
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
  !git clone https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator.git
  !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git
  # !git clone https://github.com/adieyal/sd-dynamic-prompts
  # !git clone https://github.com/kousw/stable-diffusion-webui-daam
  # !git clone 

  # LoRA  
  !git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' /content/lora/lora_diffusion/xformers_utils.py && pip install /content/lora
  !pip install accelerate bitsandbytes
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks

def set_default_prompt_and_neg():
  # TODO
  pass

# Install tool: stable-diffusion-webui
repo = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
!git clone $repo

# time machine: commit or branch
if version:
  !git checkout $version

install_dep()

use_cuda()

install_extentions()


# Launch
Provide a directory to models (sub dir is optional) and then run this script. You will get something like xxx.gradio.app, click it to go to the deployed Web UI.

---

This script use some parameters to run Web UI. The command line parameters are available at [stable-diffusion-webui Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings)

The following parameters are the most important ones for use via colab & gradio:
  - `--share` - create an online gradio.app link
  - `--gradio-debug` - print output to console
  - `--gradio-auth YOUR_ACCOUNT:YOUR_PWD` - create a login account

*   In order to prevent others from crawling your online deployment for use, it is strongly recommended to modify the username and password, and `replace account & password with others`.

---

Common Negative Prompts:
> lowres, blurry, worst quality, low quality, normal quality,bad anatomy, disfigured, deformed, mutation, mutilated, ugly, totem pole,poorly drawn face,  cloned face, several faces, long neck, mutated hands, bad hands, poorly drawn hands,extra limbs, malformed limbs, missing arms, missing fingers, extra fingers, fused fingers, too many fingers,missing legs, extra legs, malformed legs, extra digit, fewer digits, glitchy, cropped, jpeg artifacts, signature, watermark, username, text, error

In [ ]:
#@title Launch
%cd {tool_dir}

# security
account = "YOUR_ACCOUNT" #@param {type:"string"}
pwd = "YOUR_PWD" #@param {type:"string"}

# model selection
# the model dir is required to help tool to find models
model_dir = "/content/drive/MyDrive/marbel_models" #@param {type:"string"}

# the sub model dir is optional, default to use all models
sub_model_dir = "All" #@param ["All", "女巫", "融合 root", "融合 Akari H", "融合 青葉"]
if sub_model_dir == "All":
  sub_model_dir = ""
elif sub_model_dir == "女巫":
  sub_model_dir = "大理石流-統合召喚/沉默的小鎮"
elif sub_model_dir == "融合 root":
  sub_model_dir = "融合"
elif sub_model_dir == "融合 青葉":
  sub_model_dir = "融合/青葉"
elif sub_model_dir == "融合 Akari H":
  sub_model_dir = "融合/'Akari H'"
else:
  pass

# setup model dir
import os
model_dir = os.path.abspath(os.path.join(sub_model_dir, model_dir))

# use param to run tool
# notice: some param are only available at later versions
arg = f"--xformers --share --gradio-debug --gradio-auth {account}:{pwd} --disable-safe-unpickle --ckpt-dir {model_dir}"
!COMMANDLINE_ARGS="{arg}" REQS_FILE="requirements.txt" python launch.py
